In [3]:
!pip install -q pyarrow==14.0.0  cohere cassio datasets openai langchain tiktoken PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.3 MB/s eta 0:00:00


In [9]:
!pip install -q python-dotenv

In [11]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from datasets import load_dataset
from dotenv import load_dotenv
import cassio
import os
load_dotenv()

True

In [7]:
from PyPDF2 import PdfReader

In [20]:
ASTRA_DB_APPLICATION_TOKEN = < Your Astra DB Token>
ASTRA_DB_ID = <Astra DB ID>
OPENAI_API_KEY = < Your OpenAI key>

In [13]:
pdf_reader = PdfReader('NGA54097 2.pdf')

In [14]:
from typing_extensions import Concatenate

raw_text = ''
for i, page in enumerate(pdf_reader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

' \nConstitution of the Federal Republic of Nigeria \n1999  \n  \n  \n  \n Arrangement of sections  \n  \n  \nChapter I  \n  \nGeneral Provisions  \n  \n  \n  \nPart I  \nFederal Republic of Nigeria \n  \n1.  Supremacy of constitution.  2 The Federal Republic of \nNigeria.  3 States of the Federation and the \nFederal Capital Territory, Abuja.  \n  \nPart II  \n  \nPowers of the Federal Republic of Nigeria  \n  \n4.  Legislative powers.  5 Executive powers.  6 Judicial powers  \n7.  Local government system.  8 New states and boundary \nadjustment, etc.  9 Mode of altering provisions of the \nconstitution  \n10.  Prohibition of State Religion.  11  Public order and public security.  12  Implementation of treaties.  \n  \n  \nChapter II  \n  \nFundamental Objectives and directive Principles of State Policy  \n  \n13.  Fundamental obligations of the \nGovernment.  14  The Government and the people  15  Political objectives.  \n16.  Economic objectives.  17  Social objectives.  18  Educati

In [16]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(139216004386672) a86cfaa0-ffc1-4877-84ad-b200d43fcda3-us-east1.db.astra.datastax.com:29042:72433f52-bc9f-4897-8ad4-03daf2b908c9> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [22]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [24]:
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="qa_mini",
    session=None,
    keyspace=None
)

In [27]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator= "\n",
    chunk_size= 800,
    chunk_overlap=200,
    length_function = len
)

texts = text_splitter.split_text(raw_text)

In [ ]:
astra_vector_store.add_texts(texts)

In [37]:
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [39]:
question = input("Ask me any question?")
answer = astra_vector_index.query(question, llm=llm)
print(f"Answer:\n {answer}")

Ask me any question?What are the issues that can cause an annulment of election


Answer:
  There are several issues that can cause an annulment of an election, including:

- Being employed in the public service of the Federation or any State and not resigning at least 30 days before the election
- Being a member of a secret society
- Being indicted for embezzlement or fraud by a Judicial Commission of Inquiry or an Administrative Panel of Inquiry
- Presenting a forged certificate to the Independent National Electoral Commission
- Being adjudged to be a lunatic
- Failing to have a majority of YES votes over NO votes and at least one-quarter of the votes in each of two-thirds of all local government areas in the State
- Failing to have the highest number of votes and at least one-quarter of all votes in each of two-thirds of all local government areas in the State (in the case of multiple candidates)
- Other candidates being nominated after the deadline set by the Independent National Electoral Commission
- Violation of election laws and regulations, including briber

0.8906586497884152 (g)  being a person employed in the public service of t he Federation or of any Stat
0.8905252409017161 where, being the only candidate nominated for the e lection-  
(a)  he has a majorit
0.8899130688686409 other candidates, the Independent National Electora l Commission shall extend the ti
0.8893686563550165 The National Assembly shall make provisions in res pects -  
 persons who may apply 
